# figure s7

- kernel: r_env, R 4.1.3
- date: 2024-0-22

## load

In [ ]:
library(tidyverse)
library(logging)
library(ggpubr)
library(ggsci)
library(patchwork)

source('../scripts/r_funcs.r')

theme_set(theme_pubr())
logging::basicConfig()
options(warn = -1)

outdir <- '../figures/figs7'
create_dir(outdir)

In [36]:
f_pat_gp <- '../tables/patient_info_v2.tsv'
gp <- 'response'
gp_lvls$response <- c('R', 'PR')
gp_comp_map$response <- list(
  c('R-pre', 'R-post'),
  c('PR-pre', 'PR-post'),
  c('R-pre', 'PR-pre'),
  c('R-post', 'PR-post')
)
gp_comp_diff_map$response <- list(c('R', 'PR'))
gp_comp_map_pre$response <- list(c('R-pre', 'PR-pre'))
comb_order <- c('R-pre', 'R-post', 'PR-pre', 'PR-post')

In [37]:
celltype_map <- list(
    'T_MKI67' = c('T_Prolif')
)

cell_state_map <- list(
    'cytotoxic' = c('CD8_ANXA1', 'CD8_CCL5', 'CD8_CX3CR1', 'CD8_FOS', 'CD8_GZMK', 'CD8_KLRB1'),
    'exhausted' = c('CD4_CXCL13', 'CD8_CXCL13', 'CD8_TYMS', 'T_MKI67', 'Treg_TNFRSF4'),
    'dying' = c('T_Mito'),
    'others' = c('CD4_KLRB1', 'CD8_CD74', 'CD8_IFIT1', 'T_IL7R', 'T_Ribo', 'Treg_LTB')
)
cell_state_order <- c('cytotoxic', 'exhausted', 'dying', 'others')
cell_state_color <- c('exhausted' = '#023fa5', 'cytotoxic' = 'red', 
                      'dying' = '#bb7784', 'others' = '#ff9639')

## s7a: TCR sharing diff (R vs PR)

In [ ]:
f_node <- '../figures/fig3/fig3b-net_tcr_sharing-node_info.tsv'
f_edge_r_pre <- '../../stage4/a03_tcr/net_vlz_old/net_good_Baseline_edge.csv'
f_edge_r_post <- '../../stage4/a03_tcr/net_vlz_old/net_good_Treat_edge.csv'
f_edge_pr_pre <- '../../stage4/a03_tcr/net_vlz_old/net_poor_Baseline_edge.csv'
f_edge_pr_post <- '../../stage4/a03_tcr/net_vlz_old/net_poor_Treat_edge.csv'

# node info
df_node <- read_tsv(f_node, show_col_types = F) %>% 
    mutate(node_type = factor(node_type, levels = cell_state_order))

In [ ]:
# pre: R vs PR
df <- cal_edge_weight_diff(a = read_csv(f_edge_r_pre, show_col_types = F), 
                           b = read_csv(f_edge_pr_pre, show_col_types = F)) %>% 
    mutate(wtype = if_else(weight.diff > 0, 'R > PR', 'R < PR'),
           comp_type = 'Pre-treatment: R vs PR')
# add post R vs PR
df <- cal_edge_weight_diff(a = read_csv(f_edge_r_post, show_col_types = F), 
                           b = read_csv(f_edge_pr_post, show_col_types = F)) %>% 
    mutate(wtype = if_else(weight.diff > 0, 'R > PR', 'R < PR'),
           comp_type = 'Post-treatment: R vs PR') %>%
    rbind.data.frame(df)
df %>% write_tsv(str_glue('{outdir}/figs7d-edge_info-r_vs_pr.tsv'))

2024-02-22 18:12:38 INFO::a(25) - b(30) => 31 diff edges.
2024-02-22 18:12:38 INFO::a(24) - b(45) => 47 diff edges.


In [ ]:
df <- read_tsv(str_glue('{outdir}/figs7d-edge_info-r_vs_pr.tsv'), show_col_types = F)
p_ls <- lapply(X = c('Pre-treatment: R vs PR', 'Post-treatment: R vs PR'), FUN = function(ctype) {
    df_edge <- filter(df, comp_type == ctype)
    p <- exhau_cyto_netplot2(df_node = df_node, df_edge = df_edge, pt_fill = 'node_type', edge_color = 'wtype') +
        scale_fill_manual(values = cell_state_color) +
        labs(fill = 'Cell state', title = ctype, color = '') +
        theme_pubr(base_size = 14) +
        theme(plot.title = element_text(hjust = 0.5),
              legend.position = 'right', 
              legend.text = element_text(size = 14),
              legend.title = element_text(size = 14))
    return(p)
})
p_ls[[1]] <- p_ls[[1]] + theme(legend.position = 'none')
ggsave(filename = str_glue('{outdir}/figs7d-net_tcr_sharing-r_vs_pr.pdf'),
       plot = wrap_plots(p_ls), width = 18, height = 6)

## s7b: select clones in umap space

### prepare

In [ ]:
# include all RNA cells (non-TCR-RNA shared cells with NA clonotype)
f_expand <- '../../stage4/a03_tcr/expansion/clone_expand_info_per_cell.csv'
f_obj <- '../../stage4/a01_data/seuobj/t.rds'

In [ ]:
scrna <- readRDS(f_obj)

In [ ]:
# rename cell types
scrna$celltype <- scrna$subtype
for (nm in names(celltype_map)) {
    scrna$celltype[scrna$subtype %in% celltype_map[[nm]]] <- nm
}
table(scrna$celltype)
# rename sample type
scrna$timepoint <- scrna$sample_type
scrna$timepoint[scrna$sample_type == 'Baseline'] <- 'pre'
scrna$timepoint[scrna$sample_type == 'Treat'] <- 'post'
scrna$timepoint <- factor(scrna$timepoint, levels = c('pre', 'post'))


  CD4_CXCL13    CD4_KLRB1    CD8_ANXA1     CD8_CCL5     CD8_CD74   CD8_CX3CR1 
       13152        20635        15789        17401         4820         5319 
  CD8_CXCL13      CD8_FOS     CD8_GZMK    CD8_IFIT1    CD8_KLRB1     CD8_TYMS 
       32900        20473        24466         5853         5067         8936 
          NK    NK_CX3CR1       T_IL7R       T_Mito      T_MKI67       T_Ribo 
       12992         5385        27242        16101        11065         8102 
    Treg_LTB Treg_TNFRSF4 
       15682        30645 

In [ ]:
cell_clone_info <- read_csv(f_expand, show_col_types = F) %>%
    filter(!is.na(clonotype))  # RNA & TCR cells
loginfo('total %g RNA & TCR cells', nrow(cell_clone_info))

2024-02-28 20:42:52 INFO::total 205824 RNA & TCR cells


### contracted exhausted, R

span more to less

In [ ]:
show_ctypes <- c('T_MKI67', 'CD8_CXCL13', 'CD8_TYMS', 'CD8_CXCL13', 'T_Mito')
select_clone <- 'TRAV12-2;TRBV9_;TRBD1_TRAJ23;TRBJ1-5_TRA:CAVTDNQGGKLIF;TRB:CASSVTGGRGQPQHF_P21'
title_str <- 'Clone 1 (R, contracted in exhausted clusters)'

In [ ]:
cells <- cell_clone_info %>% 
    filter(clonotype == select_clone) %>% 
    pull(barcode)
loginfo('total %g cells of selected clone', length(cells))

2024-02-28 20:59:46 INFO::total 32 cells of selected clone


In [ ]:
scrna$mylabel <- scrna$celltype
scrna$mylabel[!(scrna$celltype %in% show_ctypes)] <- ' '
p <- DimPlot(object = scrna, cells.highlight = cells, group.by = 'mylabel', split.by = 'timepoint', label = T) 
p$data$celltype <- scrna@meta.data[rownames(p$data), 'celltype']
p <- p + 
    geom_point(data = p$data[cells, ], aes(UMAP_1, UMAP_2, color = celltype), size = 2, alpha = 0.8) +
    scale_color_manual(values = ctype_colors, na.value = 'lightgray') +
    ggtitle(title_str) +
    theme(legend.position = 'none')
ggsave(filename = str_glue('{outdir}/fig4g-exhau_contract-R-umap-selected_clone.pdf'), 
       plot = p, width = 7, height = 4)

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


### contracted exhausted, PR

In [ ]:
show_ctypes <- c('T_MKI67', 'CD8_CXCL13', 'CD8_TYMS')
select_clone <- 'TRAV27;TRBV20-1_;TRBD2_TRAJ49;TRBJ2-3_TRA:CAVFTGNQFYF;TRB:CSALAGGIDTQYF_P03'
title_str <- 'Clone 2 (PR, contracted in exhausted clusters)'

In [ ]:
# get cells of selected clones
cells <- cell_clone_info %>% 
    filter(clonotype == select_clone) %>% 
    pull(barcode)
loginfo('total %g cells of selected clone', length(cells))

2024-02-28 21:00:19 INFO::total 113 cells of selected clone


In [ ]:
scrna$mylabel <- scrna$celltype
scrna$mylabel[!(scrna$celltype %in% show_ctypes)] <- ' '
p <- DimPlot(object = scrna, cells.highlight = cells, group.by = 'mylabel', split.by = 'timepoint', label = T) 
p$data$celltype <- scrna@meta.data[rownames(p$data), 'celltype']
p <- p + 
    geom_point(data = p$data[cells, ], aes(UMAP_1, UMAP_2, color = celltype), size = 2, alpha = 0.8) +
    scale_color_manual(values = ctype_colors, na.value = 'lightgray') +
    ggtitle(title_str) +
    theme(legend.position = 'none')
ggsave(filename = str_glue('{outdir}/fig4g-exhau_contract-PR-umap-selected_clone.pdf'), 
       plot = p, width = 7, height = 4)

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


### expanded cytotoxic, R

In [ ]:
show_ctypes <- c('CD8_ANXA1', 'CD8_CCL5', 'CD8_CX3CR1', 'CD8_FOS', 'CD8_GZMK')
select_clone <- 'TRAV12-1;TRBV9_;_TRAJ39;TRBJ2-6_TRA:CVVNHAGNMLTF;TRB:CASSVEASGANVLTF_P26'
title_str <- 'Clone 3 (R, expanded in cytotoxic clusters)'

In [ ]:
# get cells of selected clones
cells <- cell_clone_info %>% 
    filter(clonotype == select_clone) %>% 
    pull(barcode)
loginfo('total %g cells of selected clone', length(cells))

2024-02-28 21:00:25 INFO::total 90 cells of selected clone


In [ ]:
scrna$mylabel <- scrna$celltype
scrna$mylabel[!(scrna$celltype %in% show_ctypes)] <- ' '
p <- DimPlot(object = scrna, cells.highlight = cells, group.by = 'mylabel', split.by = 'timepoint', label = T) 
p$data$celltype <- scrna@meta.data[rownames(p$data), 'celltype']
p <- p + 
    geom_point(data = p$data[cells, ], aes(UMAP_1, UMAP_2, color = celltype), size = 2, alpha = 0.8) +
    scale_color_manual(values = ctype_colors, na.value = 'lightgray') +
    ggtitle(title_str) +
    theme(legend.position = 'none')
ggsave(filename = str_glue('{outdir}/fig4g-cyto_expand-R-umap-selected_clone.pdf'), 
       plot = p, width = 7, height = 4)

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


### expanded cytotoxic, PR

In [ ]:
show_ctypes <- c('CD8_ANXA1', 'CD8_CCL5', 'CD8_FOS', 'CD8_GZMK', 'CD8_CXCL13', 'CD8_TYMS', 'T_MKI67')
select_clone <- 'TRAV20;TRBV4-2_;_TRAJ43;TRBJ2-7_TRA:CAVRGDNNDMRF;TRB:CASSHGTYSYEQYF_P20'
title_str <- 'Clone 4 (PR, expanded in cytotoxic clusters)'

In [ ]:
# get cells of selected clones
cells <- cell_clone_info %>% 
    filter(clonotype == select_clone) %>% 
    pull(barcode)
loginfo('total %g cells of selected clone', length(cells))

2024-02-28 21:00:31 INFO::total 149 cells of selected clone


In [ ]:
scrna$mylabel <- scrna$celltype
scrna$mylabel[!(scrna$celltype %in% show_ctypes)] <- ' '
p <- DimPlot(object = scrna, cells.highlight = cells, group.by = 'mylabel', split.by = 'timepoint', label = T) 
p$data$celltype <- scrna@meta.data[rownames(p$data), 'celltype']
p <- p + 
    geom_point(data = p$data[cells, ], aes(UMAP_1, UMAP_2, color = celltype), size = 2, alpha = 0.8) +
    scale_color_manual(values = ctype_colors, na.value = 'lightgray') +
    ggtitle(title_str) +
    theme(legend.position = 'none')
ggsave(filename = str_glue('{outdir}/fig4g-cyto_expand-PR-umap-selected_clone.pdf'), 
       plot = p, width = 7, height = 4)

Rasterizing points since number of points exceeds 100,000.
To disable this behavior set `raster=FALSE`

Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
